In [44]:
cd ..

/Users/jwen/Stanford/projects/smoke_segmentation


In [45]:
cd src

/Users/jwen/Stanford/projects/smoke_segmentation/src


In [46]:
import netCDF4
import numpy as np
import json
import time
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import requests
import boto3
import os

import geopandas as gpd
from datetime import datetime, timezone
from tqdm.notebook import tqdm
from shapely.geometry import Point, Polygon, box

from utils import data_prep as dp
from utils import data_downloader as dd
from utils import data_vis as dv
from utils import helpers as h

%matplotlib inline

In [47]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
# define file paths
DATA_FILE_PATH = '../data/'
NETCDF_FILE_PATH = DATA_FILE_PATH + 'netcdf/'
TEMP_NETCDF_FILE_PATH = DATA_FILE_PATH + 'temp_netcdf/'

# Process Smoke Plumes 

## Reading Smoke Plume Data and State Shapes

In [121]:
# read in the smoke plume data
result = h.get_plume_df(DATA_FILE_PATH + 'raw_smoke_plumes/')

In [50]:
# format plumes by creating intermediate features
result = dp.format_plume_data(result)

In [51]:
#result.to_file(DATA_FILE_PATH + "smoke_plumes/all_us_plumes_2018-2020.geojson", driver='GeoJSON')
result.tail()

,Density,End,Satellite,Start,geometry,year,month,day,start_doy,end_doy,start_time,end_time,view,doy,time,conus_time
104904,5.0,2022366 2350,GOES-EAST,2021001 0000,"POLYGON ((-114.45378 31.81011, -114.55632 31.7...",2021,01,01,001,001,0000,2350,F,001,2350,2351
104905,5.0,2022366 2350,GOES-EAST,2021001 0000,"POLYGON ((-114.77360 32.97710, -114.92009 32.9...",2021,01,01,001,001,0000,2350,F,001,2350,2351
104906,16.0,2022366 2100,GOES-EAST,2022366 1800,"POLYGON ((-98.65591 19.11097, -98.61574 19.110...",2023,01,01,001,001,1800,2100,F,001,2100,2101
104907,16.0,2022366 2100,GOES-EAST,2022366 1800,"POLYGON ((-114.95725 32.66393, -114.87208 32.6...",2023,01,01,001,001,1800,2100,F,001,2100,2101
104908,16.0,2022366 2350,GOES-EAST,2021001 0000,"POLYGON ((-103.24236 20.45962, -103.21490 20.4...",2021,01,01,001,001,0000,2350,F,001,2350,2351


In [52]:
## filter out empty plumes
plumes_df = result
plumes_df = plumes_df[~plumes_df.is_empty]

## filter out plumes where year is in the future... (2020-12-31 has weird data prob cause of 366 days)
plumes_df = plumes_df[plumes_df['year'].isin(['2018','2019','2020'])]

In [53]:
## check if any dropped
f"Dropped {result.shape[0] - plumes_df.shape[0]} plume observations" if plumes_df.shape != result.shape else "No plumes dropped!"

'Dropped 42 plume observations'

In [54]:
# only keep GOES east since satellite data is from goes east
plumes_df = plumes_df[plumes_df['Satellite'] == 'GOES-EAST'].reset_index(drop=True)
print(f"Number of plumes {plumes_df.shape[0]}")
plumes_df.head()

Number of plumes 93923


,Density,End,Satellite,Start,geometry,year,month,day,start_doy,end_doy,start_time,end_time,view,doy,time,conus_time
0,5.0,2018001 2312,GOES-EAST,2018001 2002,"POLYGON ((-93.79032 29.78233, -93.82389 29.535...",2018,01,01,001,001,2002,2312,C,001,2312,2312
1,5.0,2018001 2312,GOES-EAST,2018001 2002,"POLYGON ((-95.67021 33.22471, -95.67936 33.133...",2018,01,01,001,001,2002,2312,C,001,2312,2312
2,5.0,2018001 2312,GOES-EAST,2018001 2002,"POLYGON ((-97.42329 34.79477, -97.50874 34.731...",2018,01,01,001,001,2002,2312,C,001,2312,2312
3,5.0,2018001 2312,GOES-EAST,2018001 2002,"POLYGON ((-94.65153 37.89886, -94.63322 37.840...",2018,01,01,001,001,2002,2312,C,001,2312,2312
4,5.0,2018001 2312,GOES-EAST,2018001 2002,"POLYGON ((-94.76261 37.20454, -94.80412 37.138...",2018,01,01,001,001,2002,2312,C,001,2312,2312


## Get test dataset
* just for 2019 and 2020??

In [32]:
## temp plume data
temp_plumes_df = plumes_df[(plumes_df['year'].isin(['2019', '2020']))]

## may-dec
# temp_plumes_df = plumes_df[(plumes_df['year'].isin(['2019', '2020'])) & \
#                            (plumes_df['conus_time'].str.slice(0,2)>='12') & (plumes_df['conus_time'].str.slice(0,2)<='23') & \
#                            (plumes_df['month'] > '04')]

states_list = ['California', 'Nevada']
temp_plumes_df_ca_nv = h.filter_state_plumes(temp_plumes_df, states_list)

## check how many unique year, doy, and conus_time steps there are (figure out how many images there will need to be)
test = temp_plumes_df['year'] + '_' + temp_plumes_df['doy'] + '_' + temp_plumes_df['conus_time']
print(f'US plumes: {len(set(test.to_numpy()))}')

## check how many unique year, doy, and conus_time steps there are (figure out how many images there will need to be)
test = temp_plumes_df_ca_nv['year'] + '_' + temp_plumes_df_ca_nv['doy'] + '_' + temp_plumes_df_ca_nv['conus_time']
print(f'CA NV plumes: {len(set(test.to_numpy()))}')

US plumes: 2121
CA NV plumes: 493


* create dataframe with the right columns for downloading images (dont actually need to use plumes)

In [118]:
## create date range
time_df = pd.DataFrame({'date':pd.date_range(start='5/1/2019', end='10/31/2020', freq='D')})

## get necessary variables for downloading goes
time_df['year'] = pd.to_datetime(time_df['date']).dt.year.astype(str)
time_df['month'] = pd.to_datetime(time_df['date']).dt.month.astype(str).str.zfill(2)
time_df['day'] = pd.to_datetime(time_df['date']).dt.day.astype(str).str.zfill(2)
time_df['doy'] = pd.to_datetime(time_df['date']).dt.dayofyear.astype(str).str.zfill(3)
time_df['view'] = 'C'

## filter to may-oct (for compute reasons....)
time_df = time_df[(time_df.month > '04') & (time_df.month < '11')]
time_df = time_df.apply(lambda row: row.repeat(3), axis=0).reset_index(drop=True)

## add in the hour and min
time_df['conus_time'] = pd.Series(np.tile(['1601', '1901', '2201'], len(time_df)//3)).astype(str)

In [119]:
time_df.to_csv(DATA_FILE_PATH + "smoke_plumes/ca-nv_test_2019-2020.csv")

In [120]:
time_df

,date,year,month,day,doy,view,conus_time
0,2019-05-01,2019,05,01,121,C,1601
1,2019-05-01,2019,05,01,121,C,1901
2,2019-05-01,2019,05,01,121,C,2201
3,2019-05-02,2019,05,02,122,C,1601
4,2019-05-02,2019,05,02,122,C,1901
...,...,...,...,...,...,...,...
1099,2020-10-30,2020,10,30,304,C,1901
1100,2020-10-30,2020,10,30,304,C,2201
1101,2020-10-31,2020,10,31,305,C,1601
1102,2020-10-31,2020,10,31,305,C,1901


In [33]:
# save plumes data
temp_plumes_df_ca_nv.to_file(DATA_FILE_PATH + "smoke_plumes/ca-nv_plumes_2019-2020.geojson", driver='GeoJSON')

## Filter out smoke that covers large area...
* The segmentation mask will not work well if the smoke polygon covers the whole of US...
* Current heuristic....
    * Calculate Z-Score, filter out greater than 3SD of fire area & density == 5.0

In [172]:
from scipy import stats

## calculate area of smoke plume and the z_score/ density z_score
plumes_df['area'] = plumes_df['geometry'].apply(lambda x: x.area)
plumes_df['area_zscore'] = stats.zscore(plumes_df['area'])
plumes_df['density_area_zscore'] = plumes_df.groupby(['Density'])['area'].transform(stats.zscore)

## remove plumes with area greater than 3 SD & density == 5.0 or 3.5 SD & density == 16.0
plumes_df = plumes_df[~(((plumes_df['density_area_zscore']>3.0) & (plumes_df['Density']==5.0)) |
                        ((plumes_df['density_area_zscore']>3.5) & (plumes_df['Density']==16.0)))]

In [173]:
plumes_df.shape

(93496, 19)

In [174]:
# save plumes data
plumes_df.to_file(DATA_FILE_PATH + "smoke_plumes/us_plumes_2018-2020.geojson", driver='GeoJSON')

In [9]:
## temp plume data
temp_plumes_df = plumes_df[(plumes_df['year']=='2018') & (plumes_df['conus_time'].str.slice(0,2)>='12') & (plumes_df['conus_time'].str.slice(0,2)<='23') & (plumes_df['conus_time'].str.slice(2,4).isin(['02','01','31','32']))]

## check how many unique year, doy, and conus_time steps there are (figure out how many images there will need to be)
test = temp_plumes_df['year'] + '_' + temp_plumes_df['doy'] + '_' + temp_plumes_df['conus_time']
len(set(test.to_numpy()))

1346

## Download AWS files and process with satpy
* IMPORTANT:

    * On April 2, 2019 GOES changed the schedules for when the images were captured from :02 minute every 5 min to :01 every 5 min

In [6]:
## read in plume data
plumes_df = gpd.read_file(DATA_FILE_PATH + "smoke_plumes/us_plumes_2018-2020.geojson")

In [7]:
## temp plume data
temp_plumes_df = plumes_df[(plumes_df['conus_time'].str.slice(0,2)>='12') & (plumes_df['conus_time'].str.slice(0,2)<='23') & (plumes_df['conus_time'].str.slice(2,4).isin(['02','01','31','32']))]

In [8]:
temp_plumes_df.tail()

,Density,End,Satellite,Start,year,month,day,start_doy,end_doy,start_time,end_time,view,doy,time,conus_time,area,area_zscore,density_area_zscore,geometry
38891,16.0,2018363 2300,GOES-EAST,2018363 2000,2018,12,29,363,363,2000,2300,F,363,2300,2302,0.008713,-0.070997,-0.115698,"POLYGON ((-112.32848 29.65171, -112.25911 29.5..."
38892,16.0,2018363 2300,GOES-EAST,2018363 2000,2018,12,29,363,363,2000,2300,F,363,2300,2302,0.004566,-0.071056,-0.115766,"POLYGON ((-112.07106 30.70757, -112.05565 30.6..."
38893,16.0,2018363 2300,GOES-EAST,2018363 2000,2018,12,29,363,363,2000,2300,F,363,2300,2302,0.003476,-0.071071,-0.115783,"POLYGON ((-112.42559 30.73994, -112.40246 30.6..."
38894,16.0,2018363 2300,GOES-EAST,2018363 2000,2018,12,29,363,363,2000,2300,F,363,2300,2302,0.000669,-0.071111,-0.115829,"POLYGON ((-107.19574 28.38515, -107.19327 28.4..."
38895,16.0,2018363 2300,GOES-EAST,2018363 2000,2018,12,29,363,363,2000,2300,F,363,2300,2302,0.012881,-0.070938,-0.115631,"POLYGON ((-101.91955 20.34657, -101.88995 20.3..."


In [1]:
import dask.dataframe as ddf
temp_plumes_ddf = ddf.from_pandas(temp_plumes_df, npartitions=2)

NameError: name 'temp_plumes_df' is not defined

In [ ]:
import requests.exceptions as re
import warnings
warnings.simplefilter("ignore", (UserWarning, FutureWarning, RuntimeWarning))

start_time = time.time()

while True:
    try:
        dd.goes_download_wrapper_satpy(smoke_plume_data=temp_plumes_df,
                                       temp_data_path=TEMP_NETCDF_FILE_PATH,
                                 save_data_path=NETCDF_FILE_PATH, 
                                 extra_desc='', 
                                 bounds=(-124.5, 24.4, -66.6, 49.3), 
                                 bands=[1,2,3,7,11])
        break
    except (re.SSLError, re.ConnectionError, re.ChunkedEncodingError) as e:
        print("Connection Error. Continuing.")
        
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
import requests.exceptions as re
import warnings
warnings.simplefilter("ignore", (UserWarning, FutureWarning, RuntimeWarning))

start_time = time.time()

while True:
    try:
        dd.goes_download_wrapper_satpy(smoke_plume_data=temp_plumes_df,
                                       temp_data_path=TEMP_NETCDF_FILE_PATH,
                                 save_data_path=NETCDF_FILE_PATH, 
                                 extra_desc='', 
                                 bounds=None, 
                                 bands=[1,2,3,7,11])
        break
    except (re.SSLError, re.ConnectionError, re.ChunkedEncodingError) as e:
        print("Connection Error. Continuing.")
        
print("--- %s seconds ---" % (time.time() - start_time))

In [30]:
## remove files not at start or middle of hour and before 12 or after 23:59
cur_files = [file for file in os.listdir('../data/img/') if file[0] != '.']

In [22]:
blah = [cur_files[i].split('_s')[1].split('_e')[0][-2:] in ['02','01','31','32'] for i in range(len(cur_files))]

In [29]:
for file in cur_files:
    if file.split('_s')[1].split('_e')[0][-2:] not in ['02','01','31','32']:
        os.remove('../data/img/' + file)

In [27]:
cur_files[1].split('_s')[1].split('_e')[0][-2:] not in ['02','01','31','32']

False

In [32]:
len(cur_files)

939